In [2]:
from Bio.SeqIO.FastaIO import SimpleFastaParser
import config_readwrite as crw
import glob
import h5py
import os, sys
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns

import plot_params as pp
pp.fonts()

('sans-serif', 'Arial', 18)

In [3]:
#SAMPLE = 2
MER, NMUTS =14, 2

In [7]:
config, cfn = crw.read(os.path.join(os.path.dirname(os.getcwd()), "config.ini"))

#read
FA = config[f"common.{MER}mer.{NMUTS}mut"]['fa_fo-true']
TSV =  config[f"common.{MER}mer.{NMUTS}mut"]["tsv_fo-true"]

#write
section="sei"
crw.check(config, section)

PATH="/wynton/home/ahituv/fongsl/nullomers/data/lock/common/sei_predictions/chromatin-profiles-hdf5"

QUERY_F= os.path.join(PATH, f"key_common.{MER}mers.{NMUTS}mut.nulls.fo.pam.ext4096.*_predictions.h5")

COLNAMES = "/wynton/home/ahituv/fongsl/bin/sei-framework/seq_prediction_columns.txt"

config[section]["path"] = PATH
config[section]["colnames"] = COLNAMES

crw.write(config, cfn)

In [5]:
def readHdF5(filename, path):
    """
    read hdf5 file, dataset
    
    input
        filename (str) - name of hdf5 results
        path (str) - abs path to directory
        
    require 
        h5py
        
    method 
        1. assemble file
        2. read file
        3. get dataset

    return
        file, data
    
    """
    #1
    file = os.path.join(path, filename) 
    
    #2
    f = h5py.File(file, 'r')
    
    #3
    dset = f['data']

    return f, dset

def getIndexNames():
    
    """
    return list of 21097 index names corresponding to sei prediction features
    
    input
        none
    method 
        1. make a list for the index names
        2. open the index names file 
        3. append name to index name list
    
    return
        idxnames (list) - list of index names
    """
    
    #1
    idxnames = []
    
    #2
    IDXNAMES = "/wynton/home/ahituv/fongsl/bin/sei-framework/seq_prediction_columns.txt"
    with open(IDXNAMES, "r") as reader:
        for line in reader:
            #3
            idxnames.append(line.strip("\n"))
            
    return idxnames

def arrayToDF(dset):
    
    df = pd.DataFrame(np.vstack(dset))
    print(df.shape)
    df.columns = getIndexNames() # function get column names
    
    return df.reset_index()

def getRowNames(row_textfile, path):
    """
    return list corresponding to sample names
    """
    
     #1
    rownames = {}
    file = os.path.join(path, row_textfile) 
    
    #2
    with open(file, "r") as reader:
        for line in reader:
            
            #3
            idx, name = line.strip("\n").split("\t")
            if idx !="index":
                rownames[idx]=name
    df = pd.DataFrame(rownames.items(), columns= ["index_col", "id"])
    df["index_col"]=df["index_col"].astype(int)
    return df

# load data

## load hdf5

In [8]:
n = len(glob.glob(QUERY_F))
n

10

In [14]:
collection = {}
for n in np.arange(len(glob.glob(QUERY_F))):

    F = f"key_common.{MER}mers.{NMUTS}mut.nulls.fo.pam.ext4096.{n}_predictions.h5"
    F_LABELS = f"key_common.{MER}mers.{NMUTS}mut.nulls.fo.pam.ext4096.{n}_row_labels.txt"
    f, dset = readHdF5(F,PATH)

    df = arrayToDF(dset)

    ## get row names

    names = getRowNames(F_LABELS, PATH)

    # add to dataframe
    df = pd.merge(names, df, how="left", left_on ="index_col", right_on = "index")

    # drop the index_col name (redundant)
    df = df.drop(columns=["index_col", "index"])

    # format dataframe

    df["coor"] = df["id"].apply(lambda x: x.split("_")[0])
    df["pos"] = df["id"].apply(lambda x: x.split(".")[-1] if "." in x else None)
    df["pair_id"] = df["id"].apply(lambda x: (x.split("_")[1]).split(".")[0])

    # add kmer info
    df["nullomer"] = False 
    df.loc[df["id"].str.contains("null"), "nullomer" ] = True



    df = df.loc[df.pos.astype(str) != "None"]  # remove all the extra repeats of the same kmer control data
    collection[n]= df

(1994, 21907)
(1998, 21907)
(1998, 21907)
(1998, 21907)
(1998, 21907)
(1998, 21907)
(1998, 21907)
(1998, 21907)
(1998, 21907)
(1458, 21907)


In [15]:
# combine
df = pd.concat(collection.values())
print(df.shape)
df.head(33)

(9718, 21912)


,id,HeLa_Epithelium_Cervix|BTAF1|ID:1,H9_Embryonic_Stem_Cell_Embryo|EOMES|ID:1006,H9_Embryonic_Stem_Cell_Embryo|EP300|ID:1007,H9_Embryonic_Stem_Cell_Embryo|FOXH1|ID:1010,H9_Embryonic_Stem_Cell_Embryo|FOXH1|ID:1011,H9_Embryonic_Stem_Cell_Embryo|FOXP1|ID:1012,H9_Embryonic_Stem_Cell_Embryo|FOXP1|ID:1013,H9_Embryonic_Stem_Cell_Embryo|H2BK12ac|ID:1014,H9_Embryonic_Stem_Cell_Embryo|H2BK15ac|ID:1017,...,Urothelia (UT189)|DNase|ENCODE,WERI-Rb-1|CTCF|ENCODE,WERI-Rb-1|DNase|ENCODE,WI-38|CTCF|ENCODE,WI-38 (4OHTAM_20nM_72hr)|DNase|ENCODE,WI-38|DNase|ENCODE,coor,pos,pair_id,nullomer
1,chr17:77482474-77482674_ctrl-0.43,0.000079,0.014717,0.007110,0.001320,0.003049,0.079980,0.090706,0.000235,0.000087,...,0.258364,0.023295,0.030718,0.001406,0.068632,0.048106,chr17:77482474-77482674,43,ctrl-0,False
3,chr17:77482474-77482674_ctrl-1.43,0.000103,0.014600,0.008232,0.001291,0.003212,0.082509,0.086080,0.000240,0.000091,...,0.275807,0.026247,0.029319,0.001568,0.077084,0.053802,chr17:77482474-77482674,43,ctrl-1,False
5,chr17:77482474-77482674_ctrl-2.43,0.000069,0.012112,0.005903,0.001169,0.002733,0.070554,0.077229,0.000214,0.000081,...,0.251006,0.020388,0.024467,0.001364,0.063907,0.043438,chr17:77482474-77482674,43,ctrl-2,False
7,chr17:77482474-77482674_ctrl-3.43,0.000052,0.008468,0.003654,0.000760,0.001958,0.054221,0.059875,0.000123,0.000052,...,0.220965,0.008236,0.016953,0.000628,0.035971,0.026860,chr17:77482474-77482674,43,ctrl-3,False
9,chr17:77482474-77482674_ctrl-4.43,0.000053,0.008991,0.004116,0.000795,0.002051,0.058320,0.062670,0.000134,0.000056,...,0.214432,0.008752,0.019110,0.000672,0.040682,0.030374,chr17:77482474-77482674,43,ctrl-4,False
11,chr17:77482474-77482674_ctrl-5.43,0.000130,0.020483,0.014169,0.001828,0.004116,0.096462,0.107205,0.000434,0.000130,...,0.281699,0.034101,0.043569,0.001760,0.106600,0.074197,chr17:77482474-77482674,43,ctrl-5,False
13,chr17:77482474-77482674_ctrl-6.43,0.000052,0.013457,0.005265,0.001224,0.002770,0.081919,0.078133,0.000147,0.000061,...,0.260772,0.021756,0.029353,0.001378,0.064923,0.048093,chr17:77482474-77482674,43,ctrl-6,False
15,chr17:77482474-77482674_ctrl-7.43,0.000075,0.014487,0.006841,0.001350,0.003126,0.081020,0.081532,0.000237,0.000092,...,0.263421,0.018692,0.028128,0.001293,0.077490,0.052894,chr17:77482474-77482674,43,ctrl-7,False
17,chr17:77482474-77482674_ctrl-8.43,0.000066,0.015343,0.005474,0.001373,0.002850,0.085446,0.079787,0.000155,0.000071,...,0.284547,0.013729,0.031843,0.001042,0.093562,0.066021,chr17:77482474-77482674,43,ctrl-8,False
19,chr17:77482474-77482674_ctrl-9.43,0.000078,0.014007,0.006599,0.001320,0.003033,0.081481,0.083080,0.000220,0.000084,...,0.276663,0.018308,0.030547,0.001210,0.078793,0.056937,chr17:77482474-77482674,43,ctrl-9,False


In [26]:
df.head()

,id,HeLa_Epithelium_Cervix|BTAF1|ID:1,H9_Embryonic_Stem_Cell_Embryo|EOMES|ID:1006,H9_Embryonic_Stem_Cell_Embryo|EP300|ID:1007,H9_Embryonic_Stem_Cell_Embryo|FOXH1|ID:1010,H9_Embryonic_Stem_Cell_Embryo|FOXH1|ID:1011,H9_Embryonic_Stem_Cell_Embryo|FOXP1|ID:1012,H9_Embryonic_Stem_Cell_Embryo|FOXP1|ID:1013,H9_Embryonic_Stem_Cell_Embryo|H2BK12ac|ID:1014,H9_Embryonic_Stem_Cell_Embryo|H2BK15ac|ID:1017,...,Urothelia (UT189)|DNase|ENCODE,WERI-Rb-1|CTCF|ENCODE,WERI-Rb-1|DNase|ENCODE,WI-38|CTCF|ENCODE,WI-38 (4OHTAM_20nM_72hr)|DNase|ENCODE,WI-38|DNase|ENCODE,coor,pos,pair_id,nullomer
1,chr17:77482474-77482674_ctrl-0.43,0.000079,0.014717,0.007110,0.001320,0.003049,0.079980,0.090706,0.000235,0.000087,...,0.258364,0.023295,0.030718,0.001406,0.068632,0.048106,chr17:77482474-77482674,43,ctrl-0,False
3,chr17:77482474-77482674_ctrl-1.43,0.000103,0.014600,0.008232,0.001291,0.003212,0.082509,0.086080,0.000240,0.000091,...,0.275807,0.026247,0.029319,0.001568,0.077084,0.053802,chr17:77482474-77482674,43,ctrl-1,False
5,chr17:77482474-77482674_ctrl-2.43,0.000069,0.012112,0.005903,0.001169,0.002733,0.070554,0.077229,0.000214,0.000081,...,0.251006,0.020388,0.024467,0.001364,0.063907,0.043438,chr17:77482474-77482674,43,ctrl-2,False
7,chr17:77482474-77482674_ctrl-3.43,0.000052,0.008468,0.003654,0.000760,0.001958,0.054221,0.059875,0.000123,0.000052,...,0.220965,0.008236,0.016953,0.000628,0.035971,0.026860,chr17:77482474-77482674,43,ctrl-3,False
9,chr17:77482474-77482674_ctrl-4.43,0.000053,0.008991,0.004116,0.000795,0.002051,0.058320,0.062670,0.000134,0.000056,...,0.214432,0.008752,0.019110,0.000672,0.040682,0.030374,chr17:77482474-77482674,43,ctrl-4,False


# keep only cl columns

In [27]:
cl_cols = ["id", "coor","pos","pair_id","nullomer"]
CLS = ["K562", "HepG2", " WTC11"]
for cl in CLS:
    for col in df.columns:
        if cl in col:
            cl_cols.append(col)
df = df[cl_cols].drop_duplicates()

In [29]:
df.shape

(9718, 1624)

In [30]:
test_coor = set(df.loc[df['nullomer'] == True, "coor"])

print(len(test_coor))
list(test_coor)[:3]

294


['chr16:56657731-56657931',
 'chr7:45045392-45045592',
 'chr20:63879357-63879557']

# analyze one locus

In [19]:
def getPredValues(preds, track, kmer, null):

    # get raw prediction values
    kmer_pred = preds.loc[preds.index == track][kmer].iloc[0]
    null_pred = preds.loc[preds.index == track][null].iloc[0]
    ctrl_pred = preds.loc[preds.index ==
                          track][preds.columns[:-2]].T
    ctrl_pred_mean = ctrl_pred[track].mean() 
    ctrl_pred_std = ctrl_pred[track].std()

In [31]:
test.head()

,nullomer,coor,pair_id,id,pos,track,pred_prob
0,False,chr14:42147795-42147995,kmer-0,chr14:42147795-42147995_kmer-0.149,149,HeLa_Epithelium_Cervix|BTAF1|ID:1,0.000186
1,True,chr14:42147795-42147995,null-0,chr14:42147795-42147995_null-0.149,149,HeLa_Epithelium_Cervix|BTAF1|ID:1,0.000218
2,False,chr14:42147795-42147995,ctrl-0,chr14:42147795-42147995_ctrl-0.149,149,HeLa_Epithelium_Cervix|BTAF1|ID:1,0.000111
3,False,chr14:42147795-42147995,ctrl-1,chr14:42147795-42147995_ctrl-1.149,149,HeLa_Epithelium_Cervix|BTAF1|ID:1,0.000143
4,False,chr14:42147795-42147995,ctrl-2,chr14:42147795-42147995_ctrl-2.149,149,HeLa_Epithelium_Cervix|BTAF1|ID:1,0.000158


In [32]:
cldf.head()

,nullomer,coor,pair_id,id,pos,track,pred_prob
4544,False,chr14:42147795-42147995,kmer-0,chr14:42147795-42147995_kmer-0.149,149,HepG2_Epithelium_Liver|CEBPA|ID:1374,0.001156
4545,True,chr14:42147795-42147995,null-0,chr14:42147795-42147995_null-0.149,149,HepG2_Epithelium_Liver|CEBPA|ID:1374,0.000814
4546,False,chr14:42147795-42147995,ctrl-0,chr14:42147795-42147995_ctrl-0.149,149,HepG2_Epithelium_Liver|CEBPA|ID:1374,0.000607
4547,False,chr14:42147795-42147995,ctrl-1,chr14:42147795-42147995_ctrl-1.149,149,HepG2_Epithelium_Liver|CEBPA|ID:1374,0.001304
4548,False,chr14:42147795-42147995,ctrl-2,chr14:42147795-42147995_ctrl-2.149,149,HepG2_Epithelium_Liver|CEBPA|ID:1374,0.001150


In [35]:
out = os.path.join(PATH, "key_HepG2.K562.Pred.tsv")
with open(out, "w") as writer:

    v = 0
    for COOR in test_coor:


        # COOR = "chr1:10248183-10248383"
        TCOOR = '.'.join(COOR.split(":"))  # stupid formatting

        coordf = df.loc[(df["coor"] == COOR)].copy()

        # Separate by prediction positions 
        # there can be multiple places where kmer/nullomer occurs in sequence.
        for position in set(coordf["pos"]):
            cdf = coordf.loc[coordf["pos"] == position].copy()

            # melt dataframe
            test = pd.melt(cdf, id_vars=["nullomer", "coor", "pair_id", "id", "pos"],
                           var_name="track", value_name="pred_prob")

            # keep only datasets w/ controls
            if len(set(test.nullomer)) > 1:
                
                # kmer_id
                kmer = test.loc[test['pair_id'].str.contains(
                    "kmer"), "pair_id"].iloc[0]
                
                # nullomer_id
                null = test.loc[test['pair_id'].str.contains(
                    "null"), "pair_id"].iloc[0]

                # pivota dataframe so that rows are tracks, columns are experiments, and values are the predicted probabilityes
                preds = pd.pivot(test, index="pair_id", columns="track",
                                 values="pred_prob").reset_index()


                # compute dif between original kmer sequence and mutated ones.
                #dif = preds.sub(preds[kmer], axis=0).reset_index()
                dif = preds.copy()            
                for track in list(dif)[1:]:
                    dif[track] = stats.zscore(dif[track])

                    # z score for null
                    kmer_z = dif.loc[dif["pair_id"] == kmer, track].iloc[0], 
                    null_z = dif.loc[dif["pair_id"] == null, track].iloc[0]

                    # get raw prediction values
                    kmer_pred = preds.loc[preds["pair_id"]==kmer, track].iloc[0]
                    null_pred = preds.loc[preds["pair_id"]==null, track].iloc[0]

                    ctrl_pred = preds.loc[~preds["pair_id"].isin([null, kmer]), track]

                    ctrl_pred_mean, ctrl_pred_std = ctrl_pred.mean(), ctrl_pred.std()


                    # if nullomer z score is 2 sd away.

                    if abs(null_z) > 2 and abs(null_pred - ctrl_pred_mean) > 0.05:
                        print(track, null_z, abs(null_pred - ctrl_pred_mean))

                        if v<5:
                            # plot
                            fig, ax = plt.subplots(figsize=(4, 4))

                            # plot distribution of prediction differences
                            sns.histplot(preds[track])

                            # plot null difference
                            ax.axvline(null_pred, ls="--", c="r")

                            # name the plot
                            ax.set(title=track,
                                   xlabel=f"pred - {COOR}"
                                   )
                            plt.show()


                    # write data to file
                    write_info = [
                        COOR, TCOOR,
                        kmer, null,  # pair ids
                        track,  # prediction track
                        str(kmer_pred),
                        str(null_pred),
                        str(ctrl_pred_mean),
                        str(ctrl_pred_std),
                        str(null_z)  # nullomer difference predicted
                    ]

                    if v == 0:
                        columns = ["#coord", "tsvCoor", "kmer", "null",
                                   "track", "predKmer", "predNull",
                                   "predCtrl_mean", 'predCtrl_std',
                                   "difNullKmer"
                                   ]
                        writer.write("\t".join(columns)+"\n")
                        writer.write("\t".join(write_info)+"\n")
                    else:
                        writer.write("\t".join(write_info)+"\n")


                    v += 1
writer.close()

HepG2_Epithelium_Liver|CREB1|ID:46231 2.0589116285666753 0.05793886284033459
HepG2_Epithelium_Liver|CREB1|ID:56047 2.1268184864665725 0.05701528489589691
K562_Erythroblast_Bone_Marrow|H3K27ac|ID:35400 2.0220299267733535 0.08646406133969625
HepG2_Epithelium_Liver|EP300|ID:46225 -2.103716775727373 0.12586102883021033
HepG2_Epithelium_Liver|MAFK|ID:45815 -2.0751845006539797 0.06425229509671526
K562_Erythroblast_Bone_Marrow|BACH1|ID:45900 -2.0411756165069197 0.05456818739573166
K562_Erythroblast_Bone_Marrow|HMBOX1|ID:64414 -2.2580046798184985 0.10325175325075786
K562_Erythroblast_Bone_Marrow|IKZF1|ID:63463 -2.5446499926967965 0.0663925210634867
HepG2 (forskolin)|CEBPB|ENCODE 4.529944742886459 0.7126437838431836
HepG2_Epithelium_Liver|ARID3A|ID:45806 3.5073180256941896 0.1704837055876851
HepG2_Epithelium_Liver|ATF1|ID:56054 3.204844674652595 0.06508640554190302
HepG2_Epithelium_Liver|ATF4|ID:64175 4.6365861591734605 0.10085453009038853
HepG2_Epithelium_Liver|ATF4|ID:64176 4.550199162081828 

HepG2|SP1|ENCODE 2.9585995473328572 0.09441387788392604
HepG2|TEAD4|ENCODE 3.8909956249071223 0.08573444109254827
HepG2|p300|ENCODE-1 3.7711101136468574 0.11988523841525117
K562_Erythroblast_Bone_Marrow|CEBPB|ID:45923 4.7409541425787936 0.8851546519736682
K562_Erythroblast_Bone_Marrow|CEBPB|ID:46285 5.0539309122645 0.7159186374483397
K562_Erythroblast_Bone_Marrow|CEBPB|ID:63533 5.171575885879528 0.6160210794536397
K562_Erythroblast_Bone_Marrow|CEBPB|ID:63534 5.215483474139773 0.5769363862442939
K562_Erythroblast_Bone_Marrow|CEBPG|ID:64052 5.139062616446207 0.33605373301737324
K562_Erythroblast_Bone_Marrow|CEBPG|ID:64053 5.329380317695724 0.2643470099377737
K562_Erythroblast_Bone_Marrow|IKZF1|ID:63462 3.4693159549240074 0.18909057080745698
K562_Erythroblast_Bone_Marrow|IKZF1|ID:63463 3.2551327382608215 0.23212695345282555
K562|CEBPB|ENCODE 4.985040342852193 0.755376422732176
K562|CEBPB|ENCODE-1 5.367846941190349 0.22199150727683445
K562 (IFNa6h)|Pol2|ENCODE 2.169151033546986 0.063811413

K562_Erythroblast_Bone_Marrow|SIN3A|ID:46277 2.506479917509919 0.09671773190299669
K562_Erythroblast_Bone_Marrow|SMARCA4|ID:63970 2.4552343820345994 0.053111112738649054
K562_Erythroblast_Bone_Marrow|SMARCE1|ID:62879 2.3656308641644483 0.05060732718557119
K562_Erythroblast_Bone_Marrow|SUMO2|ID:55337 2.2013565933208508 0.06330796629190445
K562_Erythroblast_Bone_Marrow|SUMO2|ID:55338 2.6720383649489685 0.06121526046966513
K562_Erythroblast_Bone_Marrow|SUMO2|ID:55339 2.8413528477717866 0.05614970310901602
K562_Erythroblast_Bone_Marrow|TBP|ID:45881 2.1018246509634246 0.09970580662290256
K562_Erythroblast_Bone_Marrow|TBP|ID:50163 2.0543337533798467 0.1135740761955579
K562_Erythroblast_Bone_Marrow|ZBTB40|ID:63068 2.740834920848297 0.07345534324025114
K562_Lymphoblast|MBTD1|ID:76529 2.4973712497981695 0.07466231047486266
K562|DNase|ENCODE 2.1188735737667863 0.13251264293988546
K562|DNase|Roadmap-1 2.2559213302875394 0.1322173853715261
K562|ELF1|ENCODE 2.3876915904074427 0.06956585894028346
K5

K562_Erythroblast_Bone_Marrow|MAX|ID:45907 2.3085049390207475 0.08368550849457582
K562_Erythroblast_Bone_Marrow|MAZ|ID:45924 2.493443499263554 0.08303342688207825
K562_Erythroblast_Bone_Marrow|MTA2|ID:63517 2.584497308833159 0.06930098012089729
K562_Erythroblast_Bone_Marrow|NR4A1|ID:62815 2.587604014551003 0.06850551515817643
K562_Erythroblast_Bone_Marrow|RCOR1|ID:45903 2.4021456920748654 0.062216782259444395
K562_Erythroblast_Bone_Marrow|SMARCA4|ID:63970 2.297018680430181 0.068994030294319
K562_Erythroblast_Bone_Marrow|SMARCA4|ID:64111 2.4210775700808242 0.05609420904268821
K562_Erythroblast_Bone_Marrow|SMARCA4|ID:64112 2.3515790241867487 0.10642772217591606
K562_Erythroblast_Bone_Marrow|SMARCE1|ID:62880 2.0211156793853884 0.055414001395305
K562_Erythroblast_Bone_Marrow|SUMO2|ID:55337 2.691071313119241 0.10767289822300274
K562_Erythroblast_Bone_Marrow|SUMO2|ID:55338 2.257870584676329 0.05829685783634583
K562_Erythroblast_Bone_Marrow|TRIM24|ID:64819 2.3627361246450325 0.111795210962494

HepG2_Hepatocellular_Carcinoma|H3K4me2|Roadmap 2.5803484868501627 0.1783263514439265
HepG2_Hepatocellular_Carcinoma|H3K4me3|Roadmap 2.8405605346168468 0.1696832021077474
HepG2|ELF1|ENCODE 2.3636938232913254 0.06799653346339861
K562_Erythroblast_Bone_Marrow|ARNT|ID:62869 2.771196919618531 0.11159105574091276
K562_Erythroblast_Bone_Marrow|ATAC-seq|ID:61485 2.0173870763198267 0.1260348245501518
K562_Erythroblast_Bone_Marrow|ATF1|ID:62671 2.7542571853911126 0.11266766612728436
K562_Erythroblast_Bone_Marrow|BCOR|ID:64563 2.4455363427451355 0.14973404506842294
K562_Erythroblast_Bone_Marrow|BCOR|ID:64564 2.376348956148659 0.07724792957305908
K562_Erythroblast_Bone_Marrow|BHLHE40|ID:45916 2.074734111619272 0.13719476362069447
K562_Erythroblast_Bone_Marrow|C11orf30|ID:63363 2.7106163232133813 0.08643989339470863
K562_Erythroblast_Bone_Marrow|CBX1|ID:64916 2.0400037787635066 0.10772984921932222
K562_Erythroblast_Bone_Marrow|CEBPB|ID:46285 2.249323204460997 0.13332187458872796
K562_Erythroblast_B

...

K562_Erythroblast_Bone_Marrow|ZNF766|ID:62946 2.53236806194168 0.10193611979484557
K562_Erythroblast_Bone_Marrow|ZNF766|ID:62947 2.7482839297983914 0.07261339724063876
K562_Erythroblast_Bone_Marrow|ZSCAN29|ID:62919 3.328746052085299 0.05846562857429187
K562_Erythroblast_Bone_Marrow|ZSCAN29|ID:62920 2.51528666784337 0.07132070809602736
K562_Haematopoietic_and_lymphoid|POLR2A|ID:72442 4.329362805581296 0.10500716095169385
K562_Lymphoblast|EPC1|ID:76527 2.1393049610686594 0.05452320575714112
K562_Lymphoblast|MBTD1|ID:76529 2.0499966977573427 0.0619355022907257
K562|COREST|ENCODE 3.850841610901398 0.05107116028666496
K562|DNase|ENCODE 2.8476701209659536 0.05810894370079045
K562|DNase|Roadmap-1 3.840835188688563 0.09012863536675769
K562|ELF1|ENCODE 3.5432135661384043 0.428271280725797
K562|ETS1|ENCODE 4.197512551684245 0.07698472589254379
K562|GABP|ENCODE 3.571054629307295 0.48942702958981193
K562|H3K4me1|Roadmap -2.7625006549514053 0.08719796737035115
K562|HMGN3|ENCODE 2.2975107778066923 0

K562_Erythroblast_Bone_Marrow|KLF16|ID:64424 2.923283883469807 0.06570163485594094
K562_Erythroblast_Bone_Marrow|MTA2|ID:63518 2.363278299918932 0.10625430895015597
K562_Erythroblast_Bone_Marrow|MYNN|ID:64370 2.40173148505246 0.1272563913371414
K562_Erythroblast_Bone_Marrow|NR2F6|ID:64300 2.027308992095835 0.13005626935046166
K562_Erythroblast_Bone_Marrow|ORC2|ID:72426 2.357616263676153 0.07509602417121641
K562_Erythroblast_Bone_Marrow|POLR2A|ID:45399 2.40571225657635 0.11244911968242377
K562_Erythroblast_Bone_Marrow|POLR2A|ID:45909 2.2918875791383724 0.06643159018130973
K562_Erythroblast_Bone_Marrow|POLR2A|ID:45918 2.117633265464354 0.12385962705593556
K562_Erythroblast_Bone_Marrow|POLR2A|ID:46267 2.1179802928878217 0.17271139868535101
K562_Erythroblast_Bone_Marrow|RBBP5|ID:8795 2.024324132486366 0.05413115187548101
K562_Erythroblast_Bone_Marrow|RCOR1|ID:45911 2.2165114292946257 0.1497641144087538
K562_Erythroblast_Bone_Marrow|RNF2|ID:62638 2.007863651717274 0.1319549530162476
K562_Er

K562_Erythroblast_Bone_Marrow|BCLAF1|ID:46288 3.303155525317562 0.1547277214065675
K562_Erythroblast_Bone_Marrow|BCOR|ID:64563 2.562490338407667 0.15853333280932513
K562_Erythroblast_Bone_Marrow|BCOR|ID:64564 3.7464080754352707 0.16341961776056596
K562_Erythroblast_Bone_Marrow|BHLHE40|ID:45916 2.3080970898430633 0.06884781391389905
K562_Erythroblast_Bone_Marrow|BRD4|ID:81395 4.522994557422639 0.14704400397116138
K562_Erythroblast_Bone_Marrow|BRD4|ID:81397 4.316287624924609 0.23375096244196736
K562_Erythroblast_Bone_Marrow|CBX3|ID:46291 2.561395104190003 0.0743369075559801
K562_Erythroblast_Bone_Marrow|CDC5L|ID:62772 2.3632500764719166 0.10120963185064252
K562_Erythroblast_Bone_Marrow|CEBPA|ID:8456 5.440305190849921 0.32279055060878875
K562_Erythroblast_Bone_Marrow|CEBPB|ID:46285 2.854482045162101 0.16338851567237606
K562_Erythroblast_Bone_Marrow|CHD2|ID:45905 4.149508818676145 0.09947392248338272
K562_Erythroblast_Bone_Marrow|CSDE1|ID:63485 3.102395449980124 0.09571050636229972
K562_Er

K562_Erythroblast_Bone_Marrow|NFRKB|ID:64143 2.214861170585017 0.07574843014440225
K562_Erythroblast_Bone_Marrow|NFYA|ID:45945 3.894188080653436 0.1002673574032322
K562_Erythroblast_Bone_Marrow|NFYB|ID:45936 2.8964481615483306 0.06864162222031622
K562_Erythroblast_Bone_Marrow|NR2F1|ID:64996 3.0542437367610256 0.16098412775224258
K562_Erythroblast_Bone_Marrow|NR2F1|ID:64997 2.7993356269986975 0.13460321003390896
K562_Erythroblast_Bone_Marrow|NR2F2|ID:46265 2.5989969573648857 0.05970476519676948
K562_Erythroblast_Bone_Marrow|NR2F6|ID:64300 3.1595826696263964 0.18199370368834467
K562_Erythroblast_Bone_Marrow|NR2F6|ID:64301 4.62104207059293 0.19211643525669653
K562_Erythroblast_Bone_Marrow|NR4A1|ID:62814 2.745079035904921 0.08564257717901663
K562_Erythroblast_Bone_Marrow|NR4A1|ID:62815 2.817463260764529 0.08806086740186136
K562_Erythroblast_Bone_Marrow|ORC2|ID:72426 5.283697579911794 0.15284148171063394
K562_Erythroblast_Bone_Marrow|POLR2A|ID:45399 2.5183264038472277 0.13893867500366702
K5

K562_Lymphoblast|EPC1|ID:76527 2.64807973874561 0.14326639136960428
K562_Lymphoblast|EPC1|ID:76528 2.1695217335761927 0.06283301066967747
K562|ATF1|ENCODE 2.9444426179778036 0.14859483703490228
K562|BCL3|ENCODE 5.3592337124642215 0.09499401046383765
K562|BCLAF1|ENCODE 5.315104357380538 0.09352080800360249
K562|CBX3|ENCODE 2.8762527377995224 0.0961866609511837
K562|CCNT2|ENCODE 2.915281289796844 0.09503153254908903
K562|COREST|ENCODE 3.79866730002942 0.21772674495174038
K562|E2F4|ENCODE 3.1882081528360136 0.07478614776365222
K562|Egr-1|ENCODE 2.6799595839405 0.07282603556110012
K562|FOSL1|ENCODE 4.8450112043797 0.06843402729399743
K562|GATA2|ENCODE 5.642835186547315 0.05713983013805363
K562|HMGN3|ENCODE 4.370329327422274 0.12384977263789021
K562|JunD|ENCODE 2.120765899822979 0.05377238604330248
K562|MEF2A|ENCODE 5.315372966651828 0.12558262098220085
K562|NF-YA|ENCODE 3.6677281630221223 0.05568087749904202
K562|NR2F2|ENCODE 4.428893849015174 0.12035230883667546
K562|NRSF|ENCODE 5.2432853

In [ ]:
print(out)

# evaluate predictions

In [ ]:
cols = ['#coord',
        "tsvCoor",
        'track',
        'predKmer',
        'predNull',
        'predCtrl_mean',
        'predCtrl_std',
        'difNullKmer']
odf = pd.read_csv(out, sep='\t', 
                  usecols=cols,
                  
                 )
odf=odf.rename(columns={"#coord":"loci"})

In [ ]:
# formatting
odf=odf.loc[odf["difNullKmer"]!="difNullKmer"]

odf[['predKmer',
 'predNull',
 'predCtrl_mean',
 'predCtrl_std',
 'difNullKmer']]=odf[['predKmer',
 'predNull',
 'predCtrl_mean',
 'predCtrl_std',
 'difNullKmer']].astype(float)

# label some columns
odf["cl"] = odf["track"].apply(
    lambda x: x.split("|")[0].split("_")[0])
odf["feature"] = odf["track"].apply(lambda x: x.split("|")[1])
odf["dataset_id"] = odf["track"].apply(
    lambda x: x.split("|")[2])

odf
odf.sort_values(by="difNullKmer")#, ascending = False)

In [ ]:
features = ["H3K4me3", "H3K27ac", "H3K4me1", "H3K27me3"]
for feature in features:

    promoter = odf.loc[odf["feature"]==feature].drop_duplicates()
    print(promoter.shape)

    fig,ax=plt.subplots(figsize=(6,6))
    sns.scatterplot(x="predKmer", y='predNull', 
                    data = promoter, 
                    alpha=0.2,
                    hue="cl"
                   )
    ax.set(title=feature)
    plt.show()

# add nullomer sequences to mix

In [ ]:
test = promoter.sample(frac=0.01)
print(test.shape)
test.head()

In [ ]:
TSV= '/wynton/home/ahituv/fongsl/nullomers/data/lock/common/key_common.14mers.2mut.nulls.fo.pam.tsv'

tsv = pd.read_csv(TSV, sep='\t')

tsv.head()

In [ ]:
tsv['ends'] = tsv["matched_seq"].apply(lambda x: x[-2:])
tsv['NGG'] = tsv["ends"].apply(lambda x: True if x in ['GG', "CC"] else False)
tsv = tsv.loc[tsv.NGG == True]

In [ ]:
len(tsv.loc[tsv["NGG"]==False]), len(tsv.loc[tsv["NGG"]==True])
# lots of NGG false because kmers that do mutate to NGG can also mutate to other nullomers (not NGG nullomers.)

In [ ]:
tsv = tsv.loc[tsv["id"].str.contains('null')]
tsv.head()

In [ ]:
i = set(tsv["loci"]).intersection(set(test["loci"]))

# merge null sequence data to prediction data
plot = pd.merge(test, tsv.loc[tsv["loci"].isin(i), ["loci","matched_seq"]], 
        how="left",
        )
plot.head()

In [ ]:
fig,ax=plt.subplots(figsize=(6,6))

sns.scatterplot(x="predKmer", y='predNull', 
                data = plot, 
                hue="matched_seq",
               )
ax.set(title="H3K4me3")
ax.plot([0, 1], [0, 1], transform=ax.transAxes, ls = "--", c="grey")
#ax.legend(bbox_to_anchor=(1,1))

In [ ]:
smalldf = odf.loc[abs(odf["difNullKmer"])>1].drop_duplicates()
print(smalldf.shape)
fig, ax = plt.subplots(figsize = (4,4))
sns.scatterplot(x="predKmer", y='predNull', data = smalldf)
ax.plot([0, 1], [0, 1], transform=ax.transAxes, ls = "--", c="grey")